In [41]:
import os
import random

import pickle
import numpy as np
import scipy.io as spio
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import tensorflow as tf

from sklearn.model_selection import train_test_split

custom_seed = 42
np.random.seed(custom_seed)
tf.random.set_seed(custom_seed)

In [42]:
n_data = int(1e4)
n_discretization = 10

x_0 = y_0 = 0
x_L = y_L = 1

x = np.linspace(x_0, x_L, n_discretization).astype(np.float32)
x_step_size = (x_L - x_0) / (n_discretization - x_L)

y = np.linspace(y_0, y_L, n_discretization).astype(np.float32)
y_step_size = (y_L - y_0) / (n_discretization - y_L)

X_mesh, Y_mesh = np.meshgrid(x, y)

In [43]:
a = np.random.rand(n_data)
b = np.random.rand(n_data)
c = np.random.rand(n_data)

In [44]:
def u_func(a, b, c, X, Y):

    a = a[:, np.newaxis, np.newaxis]
    b = b[:, np.newaxis, np.newaxis]
    c = c[:, np.newaxis, np.newaxis]
    X = X[np.newaxis, :, :]
    Y = Y[np.newaxis, :, :]

    return np.sqrt(a + b*X + c*Y)

def K_func(a, b, c, X, Y):

    a = a[:, np.newaxis, np.newaxis]
    b = b[:, np.newaxis, np.newaxis]
    c = c[:, np.newaxis, np.newaxis]
    X = X[np.newaxis, :, :]
    Y = Y[np.newaxis, :, :]

    return np.sqrt(a + b*X + c*Y) * (1 - np.sqrt(a + b*X + c*Y))

def qx_func(a, b, c, X, Y):

    a = a[:, np.newaxis, np.newaxis]
    b = b[:, np.newaxis, np.newaxis]
    c = c[:, np.newaxis, np.newaxis]
    X = X[np.newaxis, :, :]
    Y = Y[np.newaxis, :, :]

    return 1/2 * b * (1 - np.sqrt(a + b*X + c*Y))

def qy_func(a, b, c, X, Y):

    a = a[:, np.newaxis, np.newaxis]
    b = b[:, np.newaxis, np.newaxis]
    c = c[:, np.newaxis, np.newaxis]
    X = X[np.newaxis, :, :]
    Y = Y[np.newaxis, :, :]

    return 1/2 * c * (1 - np.sqrt(a + b*X + c*Y))

def f_func(a, b, c, X, Y):

    a = a[:, np.newaxis, np.newaxis]
    b = b[:, np.newaxis, np.newaxis]
    c = c[:, np.newaxis, np.newaxis]
    X = X[np.newaxis, :, :]
    Y = Y[np.newaxis, :, :]

    return (1/np.sqrt(a + b*X + c*Y)) * ((b**2)/4 + (c**2)/4)

In [45]:
u = u_func(a, b, c, X=X_mesh, Y=Y_mesh)
K = K_func(a, b, c, X=X_mesh, Y=Y_mesh)
qx = qx_func(a, b, c, X=X_mesh, Y=Y_mesh)
qy = qy_func(a, b, c, X=X_mesh, Y=Y_mesh)
f_mms = f_func(a, b, c, X=X_mesh, Y=Y_mesh)

In [47]:
# Input data
X = np.concatenate(

    (u[:, :, 0, np.newaxis], 
     u[:, :, n_discretization - 1, np.newaxis],

     u[:, 0, :, np.newaxis], 
     u[:, n_discretization - 1, :, np.newaxis],

     qx[:, :, 0, np.newaxis], 
     qx[:, :, n_discretization - 1, np.newaxis],
     
     qy[:, 0, :, np.newaxis], 
     qy[:, n_discretization - 1, :, np.newaxis]),
     
axis=2)

# Ouput data
y = u

In [48]:
X_np = X
y_np = y

X_train, X_val, y_train, y_val = train_test_split(X_np, y_np, test_size=0.2, random_state=custom_seed)

a_train, a_val, b_train, b_val, c_train, c_val, = train_test_split(a, b, c, test_size=0.2, random_state=custom_seed)
qx_train, qx_val, qy_train, qy_val, u_train, u_val = train_test_split(qx, qy, u, test_size=0.2, random_state=custom_seed)
k_train, k_val, f_train, f_val = train_test_split(K, f_mms, test_size=0.2, random_state=custom_seed)

In [49]:
# Creating a dictionary to store all the data
data_dict = {
    "n_data": n_data,
    "n_discretization": n_discretization,
    "x_0": x_0,
    "x_L": x_L,
    "y_0": y_0,
    "y_L": y_L,
    "x": x,
    "x_step_size": x_step_size,
    "y": y,
    "y_step_size": y_step_size,
    "X_mesh": X_mesh,
    "Y_mesh": Y_mesh,
    "X_train": X_train,
    "X_val": X_val,
    "y_train": y_train,
    "y_val": y_val,
    "a_train": a_train,
    "a_val": a_val,
    "b_train": b_train,
    "b_val": b_val,
    "c_train": c_train,
    "c_val": c_val,
    "qx_train": qx_train,
    "qx_val": qx_val,
    "qy_train": qy_train,
    "qy_val": qy_val,
    "u_train": u_train,
    "u_val": u_val,
    "k_train": k_train,
    "k_val": k_val,
    "f_train": f_train,
    "f_val": f_val
}

# File paths
pickle_filename = "../../../data/PGNNIV_nonlinear_problem_data_creation_P3.pkl"
abs_path = os.path.abspath(pickle_filename)
dir_path = os.path.dirname(abs_path)

# Creating directory if it doesn't exist
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
    print(f"Created directory: {dir_path}")
else:
    print(f"Directory {dir_path} already exists.")

# Saving the dictionary to a pickle file
with open(pickle_filename, "wb") as f:
    pickle.dump(data_dict, f)

print(f"Data saved in directory: {dir_path}")

Directory c:\Users\usuario\Desktop\rmunozTMELab\PGNNIV_2D\data already exists.
Data saved in directory: c:\Users\usuario\Desktop\rmunozTMELab\PGNNIV_2D\data
